In [1]:
from networkit import *
import os
import numpy as np
import sys

## Read graph and coordinates

In [2]:
graphPath = "/home/harry/geographer/meshes/trace-00008.graph"
#graphPath = "/home/harry/supermuc/mairy/sofi_graph_2D_s200_vg1_fd1.graph"

In [3]:
G = readGraph(graphPath, Format.METIS)
print( G.numberOfNodes(), G.numberOfEdges() )

8993 13370


In [4]:
coordsFile = graphPath+".xyz"
coordsAll = [line.rstrip('\n') for line in open(coordsFile) if not line.startswith('%')]
print( coordsAll[:10] )

['808.000000 701.333313 0.000000', '842.666687 741.333313 0.000000', '357.333344 138.666672 0.000000', '781.333313 712.000000 0.000000', '394.666656 101.333336 0.000000', '402.666656 88.000000 0.000000', '362.666656 101.333336 0.000000', '373.333344 101.333336 0.000000', '813.333313 698.666687 0.000000', '794.666687 778.666687 0.000000']


In [5]:
coordsX = [ float(x.split(' ')[0]) for x in coordsAll ]
print( max(coordsX) )

1018.666687


In [6]:
coordsY = [ float(x.split(' ')[1]) for x in coordsAll ]
print( max(coordsY) )

1021.333313


In [7]:
#coordsZ = [ float(x.split(' ')[2]) for x in coordsAll ]
#print( max(coordsZ) )

In [8]:
S = []
j = 0
for i in range(G.numberOfNodes()):
    if coordsX[i] > 100:
        continue
    S.append(i)
G1=G.subgraphFromNodes(S).getCompactedGraph()
#G1.getCompactedGraph

AttributeError: 'networkit._NetworKit.Graph' object has no attribute 'getCompactedGraph'

## Export data to gephi

In [9]:
client = gephi.streaming.GephiStreamingClient(url="http://localhost:8080/workspace1")

In [10]:
client.exportGraph(G)

In [11]:
#client.exportNodeValues( G1, [coordsX[i] for i in S], "X")
#client.exportNodeValues( G1, [coordsY[i] for i in S], "Y")

client.exportNodeValues( G, coordsX, "X")
client.exportNodeValues( G, coordsY, "Y")

## Read partitions

In [12]:
#for a single file
direc = "/home/harry/geographer/"
tool = "ms"
filename = "trace_k6_"+tool+".part"

#partitiona = dict()

partition = [line.rstrip('\n') for line in open(direc+filename) if not line.startswith('%')]
client.exportNodeValues(G, partition, tool )

FileNotFoundError: [Errno 2] No such file or directory: '/home/harry/geographer/trace_k6_ms.part'

In [11]:
#for multiple files in same directory that start with the same prefix
direc = "/home/harry/geographer"
filename = "trace_k6_kmeans.part"
#direc = "/home/harry/supermuc/mairy/"
#filename = "sofi_graph_2D_"

partitions = dict()

for file in os.listdir(direc):
    if file.startswith(filename) and file.endswith("mtx"):
        x = file.split(".")
        x = x[0].split("_")
        #print(x)
        tool = x[-1]+"+"+x[3]+"_"+x[4]
        print(tool)
        
        partition = [line.rstrip('\n') for line in open(direc+file)]
        #first line is comments, ignore it. 
        #for the .mtx files from sofi, second line is the number of nodes; aslo ignore
        #Alternatively, we can use: ... for line in open(...) if line not startswith("%")
        partition = [ int(x) for x in partition[2:] ]
        
        partitions[tool] = partition
        

In [12]:
for x in partitions:
    client.exportNodeValues(G, partitions[x], x)

In [13]:
print( len(partitions["zoltanMJ"]))

KeyError: 'zoltanMJ'

In [54]:
# read the partitions for the different hierarchy levels

levels = 3
allHier = list()
for i in range(1,levels):
    partFile = "/home/harry/geographer-dev/partResults/partHKM10_h" + str(i) + ".out"
    allHier.append( [line.rstrip('\n') for line in open(partFile)] )
    allHier[-1] = [ int(x) for x in allHier[-1] ] 
print( allHier[0][0:10] )    
print( allHier[1][0:10] )    

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
